In [62]:
import numpy as np
import pandas as pd 

In [81]:
infered_data = pd.read_csv('./results/stride-man.csv')
original_data = pd.read_csv('./event_csv/split_data/event_csv_counts.csv')

In [82]:
infered_data.sort_values(['class', 'condition']), original_data.sort_values(['class', 'condition'])   

(     class               condition  count
 0   class2      user02_fluorescent     14
 1   class2  user02_fluorescent_led     11
 2   class2              user02_lab      9
 3   class2              user02_led     10
 4   class2          user02_natural     17
 5   class3      user02_fluorescent     13
 6   class3  user02_fluorescent_led     13
 7   class3              user02_lab     10
 8   class3              user02_led     15
 9   class3          user02_natural     16
 10  class4      user02_fluorescent      8
 11  class4  user02_fluorescent_led      8
 12  class4              user02_lab      8
 13  class4              user02_led      6
 14  class4          user02_natural     10
 15  class5      user02_fluorescent      9
 16  class5  user02_fluorescent_led      6
 17  class5              user02_lab      9
 18  class5              user02_led      6
 19  class5          user02_natural      8
 20  class6      user02_fluorescent      9
 21  class6  user02_fluorescent_led      6
 22  class6

In [83]:
# Mean absolute error, an average absolute error
def MAE(pred_count,real_count):
    return np.mean(np.abs(real_count - pred_count)/real_count)
# OffBy-One (OBO) count error.
def OBO(pred_count,real_count):
    # Predictive value and real value error
    temp = np.abs(real_count-pred_count)
    # The proportion of the prediction value of the error is less than the same
    return temp[temp<=1].shape[0]/temp.shape[0]

In [84]:
predicted_count = infered_data['count'].to_numpy()
real_count = original_data['count'].to_numpy()
MAE(predicted_count,real_count), OBO(predicted_count,real_count)

(np.float64(0.12047517297517298), 0.7)

In [85]:
infered_data[infered_data['condition'] == 'user02_fluorescent']['count']

0     14
5     13
10     8
15     9
20     9
25     5
Name: count, dtype: int64

In [86]:
event_users = [
        "user02_fluorescent",
        "user02_fluorescent_led",
        "user02_lab",
        "user02_led",
        "user02_natural"
    ]

for condition in event_users:
    predicted_count = infered_data[infered_data['condition'] == condition]['count'].to_numpy()
    real_count = original_data[original_data['condition'] == condition]['count'].to_numpy()
    mae, obo = MAE(predicted_count,real_count), OBO(predicted_count,real_count)
    print(f'{condition} MAE: {mae}, OBO: {obo}')

user02_fluorescent MAE: 0.12447089947089945, OBO: 0.8333333333333334
user02_fluorescent_led MAE: 0.1187169312169312, OBO: 0.8333333333333334
user02_lab MAE: 0.13974358974358975, OBO: 0.5
user02_led MAE: 0.15277777777777776, OBO: 0.3333333333333333
user02_natural MAE: 0.06666666666666667, OBO: 1.0


In [72]:
infered_data['count'] - infered_data['count'][0]

0     0
1    -3
2    -5
3    -4
4     3
5    -1
6    -1
7    -4
8     1
9     2
10   -6
11   -6
12   -6
13   -8
14   -4
15   -5
16   -8
17   -5
18   -8
19   -6
20   -5
21   -8
22   -5
23   -5
24   -6
25   -9
26   -7
27   -5
28   -8
29   -6
Name: count, dtype: int64

In [73]:

# True data of the original data
nature_real_count = np.load('./npy_file/nature_data_real_count.npy')
# Artificial data real count tag
artificial_real_count = np.load('./npy_file/artificial_data_real_count.npy')

# Repnet network's predictive label on original data
repnet_nature_pred_count = np.load('./npy_file/repnet_nature_data_real_count.npy')
# Repnet network's prediction label on artificial synthetic data
repnet_artificial_pred_count = np.load('./npy_file/repnet_artificial_data_real_count.npy')

In [74]:
nature_real_count, repnet_nature_pred_count

(array([15, 12, 13, 12, 17,  9,  9, 10,  8, 10, 10,  7,  9,  8,  9, 10,  8,
        10,  9,  9,  7,  8,  9,  8,  9], dtype=int32),
 array([20, 17,  7, 13, 17, 15,  8, 18, 14, 15, 12,  9, 12,  6, 10, 12, 10,
        10, 12, 14,  7, 10,  9,  8,  8], dtype=int32))

In [75]:
artificial_real_count

array([ 1, 12, 17,  3, 13, 13, 18, 18,  7,  7,  7,  1, 13, 13, 18, 13,  3,
       12, 17,  7,  7,  7], dtype=int32)

In [79]:
predicted_count = repnet_nature_pred_count
real_count = nature_real_count
MAE(predicted_count,real_count), OBO(predicted_count,real_count)

(np.float64(0.2801123321123321), 0.36)

In [ ]:
event_users = [
        "user02_fluorescent",
        "user02_fluorescent_led",
        "user02_lab",
        "user02_led",
        "user02_natural"
    ]

predicted_count_list = [] 
real_count_list = [] 
for idx, event_user in enumerate(event_users):
    predicted_count_list = [
        value for index, value in enumerate(repnet_nature_pred_count) if((index % 5) == idx)]
    real_count_list = [
        value for index, value in enumerate(nature_real_count) if((index % 5) == idx)]
    
    predicted_count = np.array(predicted_count_list)
    real_count = np.array(real_count_list)  
    mae, obo = MAE(predicted_count,real_count), OBO(predicted_count,real_count)
    print(f'{event_user} MAE: {mae}, OBO: {obo}')
    
    real_count = artificial_real_count[idx]
    mae, obo = MAE(predicted_count,real_count), OBO(predicted_count,real_count)
    print(f'{event_user} MAE: {mae}, OBO: {obo}')

user02_fluorescent MAE: 0.27999999999999997, OBO: 0.2
user02_fluorescent_led MAE: 0.2626984126984127, OBO: 0.2
user02_lab MAE: 0.31897435897435894, OBO: 0.4
user02_led MAE: 0.2833333333333333, OBO: 0.4
user02_natural MAE: 0.2555555555555556, OBO: 0.6


In [10]:
import glob 
sorted(glob.glob('./event_csv/split_data/artificial/*.csv'))

['./event_csv/split_data/artificial/a.csv',
 './event_csv/split_data/artificial/a12.csv',
 './event_csv/split_data/artificial/a17.csv',
 './event_csv/split_data/artificial/a3.csv',
 './event_csv/split_data/artificial/a3_b13.csv',
 './event_csv/split_data/artificial/a3_b13_a3.csv',
 './event_csv/split_data/artificial/a3_b18.csv',
 './event_csv/split_data/artificial/a3_b18_a3.csv',
 './event_csv/split_data/artificial/a3_b7.csv',
 './event_csv/split_data/artificial/a7.csv',
 './event_csv/split_data/artificial/a_b7_a.csv',
 './event_csv/split_data/artificial/b.csv',
 './event_csv/split_data/artificial/b13.csv',
 './event_csv/split_data/artificial/b13_a3.csv',
 './event_csv/split_data/artificial/b18.csv',
 './event_csv/split_data/artificial/b18_a3.csv',
 './event_csv/split_data/artificial/b3.csv',
 './event_csv/split_data/artificial/b3_a12_b3.csv',
 './event_csv/split_data/artificial/b3_a17_b3.csv',
 './event_csv/split_data/artificial/b7.csv',
 './event_csv/split_data/artificial/b7_a3.csv',